In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
generative = smc.ParticleTrace(1)

In [4]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [5]:
ssm_step = combinators.Model(ssm.ssm_step)
ssm_step.condition(trace=generative, observations=None)

In [6]:
for t in range(zs.shape[1] - 1):
    zs[:, t+1], mu, sigma, delta = ssm_step(zs[:, t], mu, sigma, delta, t)

In [7]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [8]:
num_particles = 100

In [9]:
smc_runner = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [10]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [11]:
inference, init_ssm_params = smc.variational_smc(num_particles, ssm.init_ssm, smc_runner, 500, 50, ssm_params, generative)

07/09/2018 14:28:08 Variational SMC ELBO=-2.78483086e+04 at epoch 1
07/09/2018 14:28:08 Variational SMC ELBO=-1.50066963e+04 at epoch 2
07/09/2018 14:28:09 Variational SMC ELBO=-1.94035234e+04 at epoch 3
07/09/2018 14:28:09 Variational SMC ELBO=-1.93765293e+04 at epoch 4
07/09/2018 14:28:10 Variational SMC ELBO=-2.24694258e+04 at epoch 5
07/09/2018 14:28:10 Variational SMC ELBO=-1.70131758e+04 at epoch 6
07/09/2018 14:28:11 Variational SMC ELBO=-2.02414082e+04 at epoch 7
07/09/2018 14:28:11 Variational SMC ELBO=-1.81666406e+04 at epoch 8
07/09/2018 14:28:12 Variational SMC ELBO=-1.70866719e+04 at epoch 9
07/09/2018 14:28:12 Variational SMC ELBO=-2.04018203e+04 at epoch 10
07/09/2018 14:28:12 Variational SMC ELBO=-1.81923945e+04 at epoch 11
07/09/2018 14:28:13 Variational SMC ELBO=-1.72661035e+04 at epoch 12
07/09/2018 14:28:13 Variational SMC ELBO=-1.92345840e+04 at epoch 13
07/09/2018 14:28:14 Variational SMC ELBO=-1.93910352e+04 at epoch 14
07/09/2018 14:28:14 Variational SMC ELBO=-1

In [12]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 1.636962
SMC MSE at time 2: 1.422140
SMC MSE at time 3: 0.060416
SMC MSE at time 4: 0.067325
SMC MSE at time 5: 0.999675
SMC MSE at time 6: 0.746684
SMC MSE at time 7: 7.612780
SMC MSE at time 8: 0.289945
SMC MSE at time 9: 0.496067
SMC MSE at time 10: 0.834456
SMC MSE at time 11: 0.147486
SMC MSE at time 12: 4.688546
SMC MSE at time 13: 0.019826
SMC MSE at time 14: 0.118246
SMC MSE at time 15: 0.373531
SMC MSE at time 16: 1.568357
SMC MSE at time 17: 1.443773
SMC MSE at time 18: 1.706828
SMC MSE at time 19: 0.295034
SMC MSE at time 20: 5.669459
SMC MSE at time 21: 2.937580
SMC MSE at time 22: 0.191948
SMC MSE at time 23: 0.510674
SMC MSE at time 24: 0.407605
SMC MSE at time 25: 4.817094
SMC MSE at time 26: 0.135761
SMC MSE at time 27: 0.708740
SMC MSE at time 28: 0.208865
SMC MSE at time 29: 1.834257
SMC MSE at time 30: 6.642456
SMC MSE at time 31: 1.313551
SMC MSE at time 32: 1.140059
SMC MSE at time 33: 0.287772
SMC MSE at time 34: 1.803255
SMC MSE at time 35: 1.5

In [13]:
samples, elbos, inference = smc.particle_mh(num_particles, ssm.init_ssm, smc_runner, 500, 50, ssm_params, generative)

In [14]:
inference

Trace{'mu': Normal([torch.cuda.FloatTensor of size 100]), 'sigma': Normal([torch.cuda.FloatTensor of size 100]), 'delta': Normal([torch.cuda.FloatTensor of size 100]), 'Z_0': Normal([torch.cuda.FloatTensor of size 100]), 'Z_1': Normal([torch.cuda.FloatTensor of size 100]), 'X_1': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_2': Normal([torch.cuda.FloatTensor of size 100]), 'X_2': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_3': Normal([torch.cuda.FloatTensor of size 100]), 'X_3': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_4': Normal([torch.cuda.FloatTensor of size 100]), 'X_4': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_5': Normal([torch.cuda.FloatTensor of size 100]), 'X_5': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_6': Normal([torch.cuda.FloatTensor of size 100]), 'X_6': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_7': Normal([torch.cuda.FloatTensor of size 100]), 'X_7': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_8': No

In [15]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [16]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.000718
SMC MSE at time 2: 3.094100
SMC MSE at time 3: 1.712189
SMC MSE at time 4: 0.362194
SMC MSE at time 5: 0.012892
SMC MSE at time 6: 0.694687
SMC MSE at time 7: 0.907699
SMC MSE at time 8: 0.066019
SMC MSE at time 9: 2.530343
SMC MSE at time 10: 0.072802
SMC MSE at time 11: 1.401813
SMC MSE at time 12: 0.005684
SMC MSE at time 13: 0.353747
SMC MSE at time 14: 0.479238
SMC MSE at time 15: 0.053064
SMC MSE at time 16: 0.203652
SMC MSE at time 17: 0.020385
SMC MSE at time 18: 0.093471
SMC MSE at time 19: 0.306279
SMC MSE at time 20: 1.355143
SMC MSE at time 21: 0.155398
SMC MSE at time 22: 4.664197
SMC MSE at time 23: 0.002347
SMC MSE at time 24: 6.281283
SMC MSE at time 25: 0.047862
SMC MSE at time 26: 0.000351
SMC MSE at time 27: 0.208756
SMC MSE at time 28: 0.014794
SMC MSE at time 29: 0.612114
SMC MSE at time 30: 0.413691
SMC MSE at time 31: 0.777103
SMC MSE at time 32: 5.000155
SMC MSE at time 33: 0.001283
SMC MSE at time 34: 0.725069
SMC MSE at time 35: 4.1